In [1]:
from pathlib import Path
from types import SimpleNamespace
import tensorflow as tf

import wandb

from data_loader import DataLoader
from utils.callbacks import TensorBoard
from utils.util import *
from utils.args_loader import load_model_config

2022-09-07 11:03:55.308351: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-07 11:03:55.308395: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data_path = Path("/mnt/disks/KITTI/small/")
data_path.exists()

True

In [3]:
arg = SimpleNamespace(model="squeezesegv2",
                      config="squeezesegv2kitti",
                      data_path=data_path,
                      train_dir="../output",
                      epochs=10)   

config, model = load_model_config(arg.model, arg.config)

2022-09-07 11:03:57.247875: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-07 11:03:57.248680: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-07 11:03:57.248749: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-09-07 11:03:57.248816: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-09-07 11:03:57.248875: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [4]:
train_dl = DataLoader("train", arg.data_path, config).write_tfrecord_dataset().read_tfrecord_dataset()
val_dl = DataLoader("val", arg.data_path, config).write_tfrecord_dataset().read_tfrecord_dataset()

TFRecord exists at /mnt/disks/KITTI/small/train.tfrecord. Skipping TFRecord writing.
TFRecord exists at /mnt/disks/KITTI/small/val.tfrecord. Skipping TFRecord writing.


## Log validation dataset

In [5]:
def resize(img, factor=1):
    return tf.image.resize(img, [img.shape[0]*factor, img.shape[1]*factor]).numpy()

In [6]:
def _create_row(lidar_input, label, class_color_map):
    "Create tuple of images and points"
    label_image = class_color_map[label.numpy().reshape(-1)].reshape([label.shape[0], label.shape[1], 3])
    depth_image = lidar_input.numpy()[:, :, [4]]
    intensity = lidar_input.numpy()[:, :, [3]]
    points = lidar_input.numpy()[:, :, :3]
    points_rgb = np.concatenate([points, (255*label_image).astype(int)], axis=-1).reshape(-1, 6)
    
    depth_image, label_image, intensity_image  = map(resize, [depth_image, label_image, intensity])
    return (wandb.Image(label_image), 
            wandb.Image(depth_image), 
            wandb.Image(intensity_image), 
            wandb.Object3D({"type": "lidar/beta", "points":points_rgb}),
           )

In [7]:
def log_input_data(lidar_input, label, class_color_map, step=None):
    "Log to media panel"
    label_image, depth_image, intensity_image, points_rgb = _create_row(lidar_input, 
                                                                        label,
                                                                        class_color_map)
    # log 2 wandb
    wandb.log({'Images/Label Image': label_image},step=step)
    wandb.log({'Images/Depth Image': depth_image},step=step)
    wandb.log({'Images/Intensity Image': intensity_image},step=step)
    wandb.log({"Images/3D": points_rgb},step=step)

In [30]:
def _create_pred_row(lidar_input, prediction, label, class_color_map):
    pred_image = class_color_map[prediction.reshape(-1)].reshape([label.shape[0], label.shape[1], 3])
    points = lidar_input.numpy()[...,:3]
    points_preds_rgb = np.concatenate([points, (255*pred_image).astype(int)], axis=-1).reshape(-1, 6)
    pred_image = resize(pred_image)
    return (wandb.Image(pred_image),
            wandb.Object3D({"type": "lidar/beta", "points":points_preds_rgb}),
           )

In [31]:
def log_model_predictions(lidar_input, prediction, label, class_color_map, step=None):
    "Log pred image and points"
    pred_image, points_rgb = _create_pred_row(lidar_input, prediction, label, class_color_map)
    wandb.log({'Images/Prediction Image':pred_image},step=step)
    wandb.log({"Images/3D_preds": points_rgb},step=step)

In [52]:
def create_pred_table(lidar_inputs,labels, predictions):
    table_data = []
    for i, (lidar_input, label, prediction) in enumerate(zip(lidar_inputs,labels, predictions)):
        input_row = _create_row(lidar_input, label, model.CLS_COLOR_MAP)
        preds_row = _create_pred_row(lidar_input, prediction, label, model.CLS_COLOR_MAP)
        table_data.append(input_row+preds_row)
    return table

In [49]:
def compute_preds(model, dataset, num_images=5):
    # a batch of images
    (lidar_inputs, lidar_masks), labels, weights = dataset.take(1).get_single_element() 

    num_images = min(num_images, lidar_inputs.shape[0])

    # select a fixed number of inputs
    lidar_inputs = lidar_inputs[:num_images, :, :]
    lidar_masks = lidar_masks[:num_images, :, :]
    labels = labels[:num_images, :, :]
    weights = weights[:num_images, :, :]

    # forward pass
    probabilities, predictions = model([lidar_inputs, lidar_masks])

    return lidar_inputs, predictions.numpy(), labels

In [50]:
lidar_inputs, predictions, labels = compute_preds(model, val_dl)

In [53]:
table = create_pred_table(lidar_inputs,labels, predictions)

In [54]:
table_data[0]

(<wandb.sdk.data_types.image.Image at 0x7f00e2e68fa0>,
 <wandb.sdk.data_types.object_3d.Object3D at 0x7f017c380250>)

In [42]:
table = wandb.Table(columns=["Label Image", "Depth Image", "Intensity Image", "LiDAR", "Pred Image", "Pred LiDAR"],
                    data=table_data)

In [43]:
with wandb.init(project="small_kitti", entity="av-team", job_type="log_preds"):
    wandb.log({"pred_val":table})

In [68]:
from wandb.keras import WandbCallback

In [ ]:
WandbCallback(save_model=